In [2]:
! pip install transformers

     |████████████████████████████████| 2.0MB 17.7MB/s 
     |████████████████████████████████| 3.2MB 48.2MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=d08b84fcab6461d09c0dbfa884d20eface76df6024e4984690acd1d41b794ef2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
import os
import torch
from transformers import BertForSequenceClassification

device = 'cuda'
dataset = 'news'

nela_path = '/gdrive/MyDrive/ECE692:MisinformationProject/data/nela-gt-2020/'
dataset_path = 'news/'
models_path = '/gdrive/MyDrive/ECE692:MisinformationProject/models'
bert_pretrained_model = 'bert-base-uncased'

import json
from dataclasses import dataclass
from transformers import BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(bert_pretrained_model)
print(os.getcwd())


/content


In [5]:
@dataclass
class TokenizedText:
    __slots__ = ['id', 'source', 'title', 'content', 'label']
    id: str
    source: str
    title: str
    content: str
    label: int

class Dataset(torch.utils.data.dataset.Dataset):
    pass

class NelaGT(Dataset):
    pass

class NelaGT2020(NelaGT):
    _nela_path = nela_path
    # _nela_path = './'
    _valid_datasets = ['news']
    _valid_subsets = ['train', 'valid']

    def __init__(self, dataset, device, subset):

        if dataset not in self._valid_datasets:
            raise ValueError(
                f"`{dataset}` is not a valid dataset. "
                f"Valid datasets are: {self._valid_datasets}"
            )

        if subset not in self._valid_subsets:
            raise ValueError(
                f"`{subset}` is not a valid subset. "
                f"Valid subsets are: {self._valid_subsets}"
            )

        torch.manual_seed(0)

        self._device = device

        self._filename = self._nela_path + f'{dataset}_{subset}.txt'

        with open(self._nela_path + f'{dataset}_{subset}_offsets.json', 'r') as f:
            self._offsets = json.load(f)

    def __len__(self):
        return len(self._offsets)

    def __getitem__(self, index):
        with open(self._filename, 'r') as f:
            f.seek(self._offsets[str(index)], os.SEEK_SET)
            line = f.readline()
            label = int(line[0])
            text = line[2:]

        return (
            tokenizer(
                text, padding='max_length', truncation=True, return_tensors="pt"
            )['input_ids'].to(self._device),
            torch.as_tensor([label], device=self._device)
        )

valid_ds = NelaGT2020(dataset, device, 'valid')

In [6]:
print(len(valid_ds))

254443


In [7]:
from torch.utils.data import SubsetRandomSampler

num_samples = 50000
random_indices = torch.randperm(num_samples)
val_sampler = SubsetRandomSampler(random_indices)

models_path = '/gdrive/MyDrive/ECE692:MisinformationProject/models/'
for filename in os.listdir(models_path):
    file_path = models_path+filename
    # print(file_path)
    model = BertForSequenceClassification.from_pretrained(file_path)
    model = model.to(device)
    valid_loader = torch.utils.data.DataLoader(valid_ds, sampler=val_sampler, batch_size=32)

    with torch.no_grad():
      correct = 0.0
      step = 1
      for X, y in valid_loader:
            outputs = model(X[0])
            logits = outputs[0]

            # loss
            criterion = torch.nn.CrossEntropyLoss()
            loss = criterion(logits, y[0]).item()


            # accuracy
            pred = torch.argmax(logits)
            correct += (pred == y[0].item()).float().item()

            #if step % 1000 == 0:
            #  print('Step', step, ':', loss, correct/step)

            step += 1
    print("%s : %f"%(file_path, correct/step))

KeyError: ignored